# Amor Reduction

## How to start:

Before starting you must:
- Have conda installed
- conda env create -f ess-notebooks-latest.yml python=3.7 . The yaml environment file is part of this repository.

## What will this notebook show?

The notebook will show how to use the `ess.amor.AmorData`, `ess.amor.AmorReference` and `ess.amor.Normalisation` classes for the reduction of data collected at the Amor instrument at PSI. 
To achieve this, we will reduce the following data files, [sample.nxs](https://github.com/scipp/ess-notebooks-data/raw/main/ess/amor/sample.nxs) and [reference.nxs](https://github.com/scipp/ess-notebooks-data/raw/main/ess/amor/references.nxs), and then normalise the sample data (with respect to the reference data). 

## Reduction

Before, we begin, we should import then necessary modules. 

In [ ]:
import os
import dataconfig
import numpy as np
import scipp as sc
import scippneutron as scn
from ess.amor import AmorData, AmorReference, Normalisation

We can then define some information about the reduction, for inclusion in the final `.ort` file.

In [ ]:
name = 'Andrew McCluskey/andrew.mccluskey@ess.eu'
affiliation = 'European Spallation Source'
data_owner = 'Jochen Stahn, PSI'
experiment_id = 'test_0001'
experiment_date = '2020-06-21'
sample_description = 'Ni-Ti Multilayer'
notebook_file = 'amor_reduction.ipynb'

The below cell enables the data to be pulled from the [`ess-notebook-data`](https://github.com/scipp/ess-notebooks-data) for the online documentation. 
For local data `data_file` should be changed to the path to the experimental file and `reference_file` to that for the reference supermirror dataset. 

In [ ]:
local_data_path = os.path.join('ess', 'amor')
data_dir = os.path.join(dataconfig.data_root, local_data_path)
data_file = os.path.join(data_dir, 'sample.nxs')
reference_file = os.path.join(data_dir, 'reference.nxs')

The `AmorData` class will take the loaded NeXus file or the NeXus file itself and perform the reduction steps to obtain the reflected intensity as a function of $q_z$, including accounting for aspects such as gravity. 
The `sample_angle_offset` allows the angular offset of the sample with respect to the horizon to be accounted for.

In [ ]:
sample = AmorData(data_file, 
                  reduction_creator=name, 
                  data_owner=data_owner, 
                  experiment_id=experiment_id, 
                  experiment_date=experiment_date, 
                  sample_description=sample_description, 
                  reduction_file=notebook_file, 
                  reduction_creator_affiliation=affiliation,
                  sample_angle_offset=0.04 * sc.units.deg, 
                  sample_size=0.1*sc.units.m)

Some detector and wavelength masking can then be performed.

In [ ]:
sample.detector_masking(y_min=0 * sc.units.m, y_max=100e-3 * sc.units.m)
sample.wavelength_masking()

The `AmorReference` class reads the reference supermirror measurement, and will perform the necessary corrections.
For this measurement, no angular offset is required. 
Again, detector and wavelength masking is performed.

In [ ]:
reference = AmorReference(reference_file)
reference.detector_masking(y_min=0 * sc.units.m, y_max=100e-3 * sc.units.m)
reference.wavelength_masking()

For the normalisation of the sample, we use the `Normalisation` class.

In [ ]:
norm = Normalisation(sample, reference)

With this object, there is the choice to bin in the $\lambda$/$\theta$-space or the $q_z$-space. 

In [ ]:
bins = (np.linspace(2.5, 15, 50), np.linspace(0.6, 1.25, 50))
lambda_theta = norm.wavelength_theta_bin(bins)

The data binned in $\lambda$/$\theta$-space can be investigated and plotted.

In [ ]:
lambda_theta

In [ ]:
sc.plot(lambda_theta, norm='log')

In [ ]:
q_bins = np.linspace(0.007, 0.09, 200)
q = norm.q_bin(q_bins)

As can be done for the $q_z$-binned data.

In [ ]:
fig = sc.plot(q, norm='log')
fig.ax.set_ylim((1e-3, 10))
fig

This data can be written to an [ORSO](https://reflectometry.org)-compatible data file. 

In [ ]:
norm.write_reflectometry('output.ort', bin_kwargs={'bins': q_bins})